### MasaCtrl: Tuning-free Mutual Self-Attention Control for Consistent Image Synthesis and Editing

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
from einops import rearrange, repeat
from omegaconf import OmegaConf

from diffusers import DDIMScheduler, ControlNetModel

from masactrl.diffuser_utils import MasaCtrlPipeline, MasaCtrlControlNetPipeline
from masactrl.masactrl_utils import AttentionBase
from masactrl.masactrl_utils import regiter_attention_editor_diffusers

from torchvision.utils import save_image
from torchvision.io import read_image
from pytorch_lightning import seed_everything

torch.cuda.set_device(0)  # set the GPU device

/mnt/hdd/hbchoe/anaconda3/envs/Masactrl2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Model Construction

In [2]:
# Note that you may add your Hugging Face token to get access to the models
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model_path = "xyn-ai/anything-v4.0"
model_path = "stable-diffusion-v1-5/stable-diffusion-v1-5"
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose")

# model = MasaCtrlPipeline.from_pretrained(model_path, scheduler=scheduler, cross_attention_kwargs={"scale": 0.5}).to(device)
model = MasaCtrlControlNetPipeline.from_pretrained(model_path, controlnet=controlnet, scheduler=scheduler, cross_attention_kwargs={"scale": 0.5}).to(device)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/mnt/hdd/hbchoe/anaconda3/envs/Masactrl2/lib/python3.8/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Keyword arguments {'cross_attention_kwargs': {'scale': 0.5}} are not expected by MasaCtrlControlNetPipeline and will be ignored.
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do 

#### Consistent synthesis with MasaCtrl

In [50]:
from masactrl.masactrl import MutualSelfAttentionControl
from PIL import Image


seed = 42
seed_everything(seed)

out_dir = "./workdir/masactrl_exp/"
os.makedirs(out_dir, exist_ok=True)
sample_count = len(os.listdir(out_dir))
out_dir = os.path.join(out_dir, f"sample_{sample_count}")
os.makedirs(out_dir, exist_ok=True)

prompts = [
    "1boy, casual, outdoors, standing",  # source prompt
    "1boy, casual, outdoors, dancing"  # target prompt
]

condition_image = "/mnt/hdd/hbchoe/workspace/MasaCtrl/dataset/poses/dance_03.png"
# load the condition image
condition_image = read_image(condition_image).float() / 255.0
# rgba to rgb conversion
if condition_image.shape[0] == 4:
    condition_image = condition_image[:3, :, :]
    # resize to 512x512
condition_image = F.interpolate(condition_image.unsqueeze(0), size=(512, 512), mode='bilinear', align_corners=False)
condition_image = condition_image.to(device)
zero_condition = torch.zeros_like(condition_image)
condition = torch.cat([zero_condition, condition_image], dim=0)  # concatenate the condition image and zero condition


# print("Condition image shape:", condition_image.shape)
# print("Zero Condition shape:", zero_condition.shape)
# print("Condition shape after concatenation:", condition.shape)



# initialize the noise map
start_code = torch.randn([1, 4, 64, 64], device=device)
start_code = start_code.expand(len(prompts), -1, -1, -1)



Seed set to 42


Condition image shape: torch.Size([1, 3, 512, 512])
Zero Condition shape: torch.Size([1, 3, 512, 512])
Condition shape after concatenation: torch.Size([2, 3, 512, 512])


In [51]:
# inference the synthesized image without MasaCtrl
editor = AttentionBase()
regiter_attention_editor_diffusers(model, editor)
image_ori = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)

DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:25<00:00,  1.98it/s]


In [52]:
# inference the synthesized image with MasaCtrl
STEP = 4
LAYPER = 10

# hijack the attention module
editor = MutualSelfAttentionControl(STEP, LAYPER)
regiter_attention_editor_diffusers(model, editor)

# inference the synthesized image
image_masactrl = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)[-1:]

# save the synthesized image
out_image = torch.cat([image_ori, image_masactrl], dim=0)
save_image(out_image, os.path.join(out_dir, f"all_step{STEP}_layer{LAYPER}.png"))
save_image(out_image[0], os.path.join(out_dir, f"source_step{STEP}_layer{LAYPER}.png"))
save_image(out_image[1], os.path.join(out_dir, f"without_step{STEP}_layer{LAYPER}.png"))
save_image(out_image[2], os.path.join(out_dir, f"masactrl_step{STEP}_layer{LAYPER}.png"))

print("Syntheiszed images are saved in", out_dir)

MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]


DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]

DDIM Sampler: 100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


Syntheiszed images are saved in ./workdir/masactrl_exp/sample_54


# Batch Generation

In [56]:
from masactrl.masactrl import MutualSelfAttentionControl
from PIL import Image


seed = 42
seed_everything(seed)

# initialize the noise map
start_code = torch.randn([1, 4, 64, 64], device=device)
start_code = start_code.expand(len(prompts), -1, -1, -1)

Seed set to 42


In [57]:
prompts = [
    "1boy, casual, outdoors, standing",  # source prompt
    "1boy, casual, outdoors, dancing"  # target prompt
]

condition_image = "/mnt/hdd/hbchoe/workspace/MasaCtrl/dataset/poses/dance_01.png"
# load the condition image
condition_image = read_image(condition_image).float() / 255.0
# rgba to rgb conversion
if condition_image.shape[0] == 4:
    condition_image = condition_image[:3, :, :]
    # resize to 512x512
condition_image = F.interpolate(condition_image.unsqueeze(0), size=(512, 512), mode='bilinear', align_corners=False)
condition_image = condition_image.to(device)
zero_condition = torch.zeros_like(condition_image)
condition = torch.cat([zero_condition, condition_image], dim=0)  # concatenate the condition image and zero condition



In [58]:
# inference the synthesized image without MasaCtrl
editor = AttentionBase()
regiter_attention_editor_diffusers(model, editor)
image_ori = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)

/mnt/hdd/hbchoe/anaconda3/envs/Masactrl2/lib/python3.8/site-packages/diffusers/models/unet_2d_condition.py:452: FutureWarning: Accessing `in_channels` directly via unet.in_channels is deprecated. Please use `unet.config.in_channels` instead
  deprecate(
DDIM Sampler: 100%|██████████| 50/50 [00:24<00:00,  2.01it/s]


In [61]:
from torchvision.transforms import ToPILImage
# Convert the PyTorch tensor to PIL image before saving
ToPILImage()(image_ori[0].cpu()).save("final_test/final_test_zero_cond_original.png")
ToPILImage()(image_ori[1].cpu()).save("final_test/final_test_zero_cond_without.png")


In [62]:
import glob
import os
from torchvision.utils import save_image

STEP = 4
LAYER = 10
# sequential generation

folder_path = "/mnt/hdd/hbchoe/workspace/MasaCtrl/dataset/poses"
output_folder = "final_test"
control_image_files = sorted(glob.glob(f"{folder_path}/*.png"))

# conditioning image preprocess
condition_image = "/mnt/hdd/hbchoe/workspace/MasaCtrl/dataset/poses/dance_03.png"



for file in control_image_files:
    # load the condition image
    condition_image = read_image(file).float() / 255.0
    # rgba to rgb conversion
    if condition_image.shape[0] == 4:
        condition_image = condition_image[:3, :, :]
        # resize to 512x512
    condition_image = F.interpolate(condition_image.unsqueeze(0), size=(512, 512), mode='bilinear', align_corners=False)
    condition_image = condition_image.to(device)
    zero_condition = torch.zeros_like(condition_image)
    condition = torch.cat([zero_condition, condition_image], dim=0)  # concatenate the condition image and zero condition

    # inference the synthesized image with MasaCtrl
    STEP = 4
    LAYPER = 10

    # hijack the attention module
    editor = MutualSelfAttentionControl(STEP, LAYPER)
    regiter_attention_editor_diffusers(model, editor)

    # inference the synthesized image
    image_masactrl = model(prompts, controlnet_conditioning=condition, latents=start_code, guidance_scale=7.5)[-1:]
    # Save the edited image
    file_name, file_ext = os.path.splitext(os.path.basename(file))
    # image_masactrl.save(f"{output_folder}/final_test_{file_name}.png")  # with attention hijack
    save_image(image_masactrl, f"{output_folder}/final_test_{file_name}.png")  # with attention hijack


/mnt/hdd/hbchoe/anaconda3/envs/Masactrl2/lib/python3.8/site-packages/diffusers/models/unet_2d_condition.py:452: FutureWarning: Accessing `in_channels` directly via unet.in_channels is deprecated. Please use `unet.config.in_channels` instead
  deprecate(


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]


DDIM Sampler: 100%|██████████| 50/50 [00:27<00:00,  1.80it/s]


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]


DDIM Sampler: 100%|██████████| 50/50 [00:28<00:00,  1.77it/s]


...

DDIM Sampler: 100%|██████████| 50/50 [00:28<00:00,  1.74it/s]


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]


DDIM Sampler: 100%|██████████| 50/50 [00:28<00:00,  1.74it/s]


MasaCtrl at denoising steps:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
MasaCtrl at U-Net layers:  [10, 11, 12, 13, 14, 15]


DDIM Sampler: 100%|██████████| 50/50 [00:28<00:00,  1.74it/s]
